In [20]:
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter, defaultdict
import pickle
import re
from tqdm.notebook import tqdm, trange

In [101]:
from dataloader import *

In [3]:
imdb = pd.read_csv('data/imdb.tsv',  sep='\t', header=0)
rating = pd.read_csv('data/ratings.tsv', sep='\t', header=0)

C:\Users\Ariel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
imdb = imdb.merge(rating, how='left', on=['tconst'])
imdb = imdb[imdb['numVotes'].notna()]
imdb = imdb[~imdb['titleType'].isin(['tvEpisode', 'tvSpecial', 'video', 'short'])]
imdb = imdb.reset_index(drop=True)

In [5]:
imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.5,8.0
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",6.1,632.0
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,6.0,5.0
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,4.5,14.0
4,tt0000630,movie,Hamlet,Amleto,0,1908,\N,\N,Drama,3.8,12.0
...,...,...,...,...,...,...,...,...,...,...,...
412029,tt9916362,movie,Coven,Akelarre,0,2020,\N,92,"Adventure,Drama,History",6.3,3218.0
412030,tt9916380,tvSeries,Meie aasta Aafrikas,Meie aasta Aafrikas,0,2019,\N,43,"Adventure,Comedy,Family",9.0,104.0
412031,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,\N,\N,"Adventure,History,War",3.8,12.0
412032,tt9916460,tvMovie,Pink Taxi,Pink Taxi,0,2019,\N,\N,Comedy,9.3,15.0


In [7]:
titles = imdb['primaryTitle'].tolist()

In [97]:
np.unique(imdb['titleType'])

array(['movie', 'tvMiniSeries', 'tvMovie', 'tvSeries', 'tvShort',
       'videoGame'], dtype=object)

In [98]:
def search_for_closest(query):
    ids = []
    for idx, t in enumerate(titles):
        if query in str(t):
            ids.append(idx)
    ranked = imdb.iloc[ids].sort_values(by='numVotes', ascending=False)
    if len(ranked) > 0:
        return ranked.iloc[0].to_dict()
    return {}

In [18]:
fandom_count = Counter(chain.from_iterable(fandoms))
fandom_total = len(fandom_count)

In [19]:
def parse_fandom(fandom):
    f = fandom.split(' | ')[-1]
    match_obj = re.match(r'([^\(\)\-]+)($|\(.+\)|- All Media Types)?', f)
    if not match_obj:
        return f
    else:
        return match_obj[1].strip(' '), match_obj[2].replace(')', '').replace('(', '') if match_obj[2] else ''

In [104]:
names = []
fandoms = fandom_df['ao3_name'].tolist()
for fandom in fandoms:
    parsed = parse_fandom(fandom)
    names.append(parsed[0])
    
# for fandom, count in tqdm(sorted(fandom_count.items(), key=lambda x:-x[1])):
    
#     res = search_for_closest(parsed[0])
#     if res:
#         res['numVotes'] = int(res['numVotes'])
#         res['genres'] = res['genres'].split(',')
#         if 'tv' in res['titleType']:
#             res['titleType'] = 'tv'
#         elif 'video' in res['titleType']:
#             res['titleType'] = 'video_game'
    
#     entry = {
#         'ao3_name': fandom,
#         'count': count,
#         'percentage': round(count / fandom_total, 4),
#         'imdb_name': res.get('primaryTitle', ''),
#         'media_type': res.get('titleType', ''),
#         'genres': res.get('genres', ''),
#         'votes': res.get('numVotes', ''),
#         'start_year': res.get('startYear', ''),
#     }
#     data.append(entry)

In [22]:
fandom_df = pd.DataFrame(data)

In [110]:
fandom_df['parsed_ao3_name'] = names

In [115]:
fandom_df.head(10)

,ao3_name,count,percentage,imdb_name,media_type,genres,votes,start_year,parsed_ao3_name
0,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,3753,0.5350,My Hero Academia,tv,"[Action, Adventure, Animation]",40037,2016,My Hero Academia
1,Video Blogging RPF,3724,0.5309,,,,,,Video Blogging RPF
2,Minecraft (Video Game),3705,0.5282,Minecraft,video_game,"[Action, Adventure, Family]",8031,2009,Minecraft
3,Marvel Cinematic Universe,2737,0.3902,,,,,,Marvel Cinematic Universe
4,Harry Potter - J. K. Rowling,2704,0.3855,Harry Potter and the Deathly Hallows: Part 2,movie,"[Adventure, Drama, Fantasy]",780949,2011,Harry Potter
5,방탄소년단 | Bangtan Boys | BTS,2612,0.3723,BTS: Burn the Stage,tv,[Documentary],1411,2018,BTS
6,Haikyuu!!,2586,0.3686,Haikyuu!!,tv,"[Animation, Comedy, Drama]",13258,2014,Haikyuu!!
7,原神 | Genshin Impact (Video Game),2066,0.2945,Genshin Impact,video_game,"[Action, Adventure, Fantasy]",368,2020,Genshin Impact
8,Star Wars - All Media Types,1502,0.2141,Star Wars: Episode IV - A New Hope,movie,"[Action, Adventure, Fantasy]",1249514,1977,Star Wars
9,Shingeki no Kyojin | Attack on Titan,1358,0.1936,Attack on Titan,tv,"[Action, Adventure, Animation]",226886,2013,Attack on Titan


In [ ]:
pickle.dump(fandom_df, open('data/fandom.p', 'wb'))

In [85]:
genres = list(zip(fandom_df['genres'].tolist(), fandom_df['media_type'].tolist()))
genres = [g1 + [g2] if g1 != '' else [] for g1, g2 in genres]

count = np.log2(np.array(fandom_df['count'].values)).reshape(-1, 1)
votes = np.log2(np.array([int(v) if v else 1e-6 for v in fandom_df['votes'].values])).reshape(-1, 1)

year = np.array([int(y) if y != '' else 0 for y in fandom_df['start_year'].values]).reshape(-1, 1)

In [86]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
G = mlb.fit_transform(genres)

G_c = np.concatenate([G, count * 0.4, year * 0.3, votes * 0.3], axis=1)

In [87]:
mlb.classes_

array(['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western', '\\N', 'movie', 'tv',
       'video_game'], dtype=object)

In [88]:
G_c[0]

array([  1.        ,   0.        ,   1.        ,   1.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   1.        ,   0.        ,
         4.74953143, 604.8       ,   4.58671388])

In [92]:
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(G_c, G_c[0].reshape(1, -1)).flatten()
fandom_df.iloc[np.argsort(-cs)[:10]]

,ao3_name,count,percentage,imdb_name,media_type,genres,votes,start_year
0,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,3753,0.5350,My Hero Academia,tv,"[Action, Adventure, Animation]",40037,2016
14,呪術廻戦 | Jujutsu Kaisen (Manga),1013,0.1444,Jujutsu Kaisen,tv,"[Action, Adventure, Animation]",16598,2020
16,呪術廻戦 | Jujutsu Kaisen (Anime),951,0.1356,Jujutsu Kaisen,tv,"[Action, Adventure, Animation]",16598,2020
22,Naruto,839,0.1196,Naruto: Shippûden,tv,"[Action, Adventure, Animation]",89552,2007
9,Shingeki no Kyojin | Attack on Titan,1358,0.1936,Attack on Titan,tv,"[Action, Adventure, Animation]",226886,2013
27,Star Wars: The Clone Wars (2008) - All Media T...,634,0.0904,Star Wars: The Clone Wars,tv,"[Action, Adventure, Animation]",67940,2008
30,ジョジョの奇妙な冒険 | JoJo no Kimyou na Bouken | JoJo's...,598,0.0852,JoJo's Bizarre Adventure,tv,"[Action, Adventure, Animation]",11469,2012
28,Avatar: The Last Airbender,633,0.0902,Avatar: The Last Airbender,tv,"[Action, Adventure, Animation]",256992,2005
54,One Piece,331,0.0472,One Piece,tv,"[Action, Adventure, Animation]",75495,1999
33,RWBY,498,0.0710,RWBY,tv,"[Action, Adventure, Animation]",5229,2012
